Question 1: What is Generative AI and what are its primary use cases across industries?

Answer:

Generative AI refers to a class of artificial intelligence models that can generate new content such as text, images, audio, video, or code by learning patterns from existing data. Unlike traditional AI systems that classify or predict outcomes, generative models create new data samples that resemble the training data.

Generative AI typically uses deep learning architectures such as Transformers, Variational Autoencoders (VAEs), Generative Adversarial Networks (GANs), and diffusion models.

Primary use cases across industries include:

Healthcare – Drug discovery, medical report generation, and synthetic data generation.

Finance – Fraud detection simulation, automated report writing, and algorithmic trading strategies.

Media & Entertainment – Script writing, music generation, video editing, and game design.

Education – Personalized tutoring systems and automated content creation.

E-commerce – Product description generation and chatbot-based customer support.

Manufacturing – Design optimization and digital twin simulations.

Generative AI enhances creativity, automation, and personalization at scale.

Question 2: Explain the role of probabilistic modeling in generative models. How do these models differ from discriminative models?

Answer:

Probabilistic modeling forms the foundation of generative models. These models learn the joint probability distribution:

P(X, Y)

or simply:

P(X)

This allows them to generate new samples by sampling from the learned distribution.

Generative models aim to model how data is generated. Examples include:

Naive Bayes

Hidden Markov Models

Variational Autoencoders

GANs

Discriminative models, on the other hand, learn:

P(Y | X)

They focus on decision boundaries between classes rather than modeling the data distribution.

Key Differences:

Generative Models	Discriminative Models
Learn data distribution	Learn decision boundary
Can generate new samples	Cannot generate new data
More complex training	Usually simpler training
Example: VAE, GAN	Example: Logistic Regression, SVM

Generative models are powerful for simulation, synthesis, and creative AI applications.

Question 3: What is the difference between Autoencoders and Variational Autoencoders (VAEs) in the context of text generation?

Answer:

Autoencoders are neural networks designed to reconstruct input data. They consist of:

Encoder: Compresses input into latent representation.

Decoder: Reconstructs input from latent representation.

They learn deterministic mappings.

Variational Autoencoders (VAEs) extend this idea using probabilistic latent variables. Instead of learning a fixed latent vector, VAEs learn:

Mean (μ) and Variance (σ²)

They sample from a Gaussian distribution:

z ~ N(μ, σ²)

Key Differences:

Autoencoder	Variational Autoencoder
Deterministic latent space	Probabilistic latent space
No sampling	Uses reparameterization trick
Not ideal for generation	Good for generation
No KL divergence	Includes KL divergence loss

In text generation, VAEs can generate diverse sentences by sampling different latent vectors.

Question 4: Describe the working of attention mechanisms in Neural Machine Translation (NMT). Why are they critical?

Answer:

Attention mechanisms allow the decoder to focus on relevant parts of the input sequence while generating each output word.

Traditional encoder-decoder models compress the entire input into a single context vector. This causes information loss for long sentences.

Attention solves this by:

Calculating alignment scores between decoder hidden state and encoder outputs.

Applying softmax to get attention weights.

Computing weighted sum of encoder outputs.

Feeding this context vector to decoder.

Why critical?

Handles long sentences better.

Improves translation accuracy.

Enables parallel computation (in Transformers).

Forms the backbone of modern models like Transformers.

Attention enables dynamic context learning instead of fixed compression.

Question 5: What ethical considerations must be addressed when using generative AI for creative content such as poetry or storytelling?

Answer:

Key ethical considerations include:

Copyright Issues – AI-generated text may resemble copyrighted material.

Bias – Models trained on biased data may produce discriminatory content.

Misinformation – Generated stories may spread false narratives.

Authenticity – Unclear ownership of AI-generated content.

Deepfake Risks – Misuse in deceptive storytelling.

Cultural Sensitivity – Offensive or culturally inappropriate outputs.

Mitigation Strategies:

Bias audits

Dataset filtering

Human-in-the-loop moderation

Transparency in AI usage

Responsible AI practices are essential for safe deployment.

Question 6. Use the following small text dataset to train a simple Variational
Autoencoder (VAE) for text reconstruction:
["The sky is blue", "The sun is bright", "The grass is green",
"The night is dark", "The stars are shining"]
1. Preprocess the data (tokenize and pad the sequences).
2. Build a basic VAE model for text reconstruction.
3. Train the model and show how it reconstructs or generates similar sentences.
Include your code, explanation, and sample outputs.



In [14]:



import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# =====================================
# 2. Dataset
# =====================================

sentences = [
    "The sky is blue",
    "The sun is bright",
    "The grass is green",
    "The night is dark",
    "The stars are shining"
]

# =====================================
# 3. Tokenization & Padding
# =====================================

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

sequences = tokenizer.texts_to_sequences(sentences)
max_len = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

vocab_size = len(tokenizer.word_index) + 1

print("Word Index:\n", tokenizer.word_index)
print("\nPadded Sequences:\n", padded_sequences)
print("\nVocabulary Size:", vocab_size)
print("Max Sequence Length:", max_len)

# =====================================
# 4. Build Encoder
# =====================================

embedding_dim = 16
latent_dim = 8

encoder_inputs = keras.Input(shape=(max_len,))
x = layers.Embedding(vocab_size, embedding_dim)(encoder_inputs)
x = layers.Flatten()(x)

z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

# Sampling layer
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = Sampling()([z_mean, z_log_var])

encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

# =====================================
# 5. Build Decoder
# =====================================

latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(max_len * vocab_size, activation="softmax")(latent_inputs)
decoder_outputs = layers.Reshape((max_len, vocab_size))(x)

decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

# =====================================
# 6. Define VAE Model (Subclassing)
# =====================================

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)

            reconstruction_loss = tf.reduce_mean(
                keras.losses.sparse_categorical_crossentropy(data, reconstruction)
            )

            kl_loss = -0.5 * tf.reduce_mean(
                1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            )

            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

# =====================================
# 7. Train VAE
# =====================================

vae = VAE(encoder, decoder)
vae.compile(optimizer="adam")

vae.fit(padded_sequences, epochs=200, batch_size=2)

# =====================================
# 8. Reconstruction
# =====================================

z_mean, z_log_var, z = encoder.predict(padded_sequences)
reconstructed = decoder.predict(z)

print("\n===== Reconstruction Results =====\n")

for i in range(len(reconstructed)):
    predicted_sequence = np.argmax(reconstructed[i], axis=1)
    words = [tokenizer.index_word.get(idx, '') for idx in predicted_sequence]

    print("Original:     ", sentences[i])
    print("Reconstructed:", " ".join(words))
    print()

Word Index:
 {'the': 1, 'is': 2, 'sky': 3, 'blue': 4, 'sun': 5, 'bright': 6, 'grass': 7, 'green': 8, 'night': 9, 'dark': 10, 'stars': 11, 'are': 12, 'shining': 13}

Padded Sequences:
 [[ 1  3  2  4]
 [ 1  5  2  6]
 [ 1  7  2  8]
 [ 1  9  2 10]
 [ 1 11 12 13]]

Vocabulary Size: 14
Max Sequence Length: 4
Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - kl_loss: 0.0012 - loss: 2.7779 - reconstruction_loss: 2.7768
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - kl_loss: 0.0011 - loss: 2.6840 - reconstruction_loss: 2.6829
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - kl_loss: 0.0011 - loss: 2.7599 - reconstruction_loss: 2.7588
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - kl_loss: 0.0011 - loss: 2.6016 - reconstruction_loss: 2.6005    
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - kl_loss: 0.0013 - loss: 2.7325 - reconstruction_loss: 2.7312
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - kl_loss: 0.0016 - loss: 2.7490 - reconstruction_loss: 2.7474
Epoch 7/200
3/

Question 7. Use a pre-trained GPT model (like GPT-2 or GPT-3) to translate a short
English paragraph into French and German. Provide the original and translated text.


In [17]:


# ============================================
# 2. Import Libraries
# ============================================

from transformers import pipeline
import torch

# ============================================
# 3. Load Pre-trained GPT-2 Model
# ============================================

generator = pipeline("text-generation", model="gpt2")

# ============================================
# 4. Define English Paragraph
# ============================================

english_text = """
Artificial Intelligence is transforming industries worldwide.
It improves efficiency, enables automation, and creates new opportunities for innovation.
"""

print("========== ORIGINAL ENGLISH TEXT ==========\n")
print(english_text)

# ============================================
# 5. Translate into French
# ============================================

prompt_french = f"""
Translate the following English text into French:

{english_text}

French Translation:
"""

french_output = generator(prompt_french, max_length=200, num_return_sequences=1)

# Extract only translated portion
french_translation = french_output[0]['generated_text'].split("French Translation:")[-1].strip()

print("\n========== FRENCH TRANSLATION ==========\n")
print(french_translation)

# ============================================
# 6. Translate into German
# ============================================

prompt_german = f"""
Translate the following English text into German:

{english_text}

German Translation:
"""

german_output = generator(prompt_german, max_length=200, num_return_sequences=1)

german_translation = german_output[0]['generated_text'].split("German Translation:")[-1].strip()

print("\n========== GERMAN TRANSLATION ==========\n")
print(german_translation)

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


========== ORIGINAL ENGLISH TEXT ==========


Artificial Intelligence is transforming industries worldwide.
It improves efficiency, enables automation, and creates new opportunities for innovation.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



========== FRENCH TRANSLATION ==========

"Artificial intelligence" is a concept that has been around for a long time. It is a term that translates to "intelligent" or "intelligent" technology that will help make us smarter, healthier, and more productive.


The most recent example of artificial intelligence in action is the United States. It has been called the "Jellybean of Artificial Intelligence", and has been used in many fields such as health and medicine.

Artificial Intelligence is transforming industries worldwide.Technology changes the world for the better. It is a concept that has been around for a long time. It is a term that translates to "intelligent" or "intelligent" technology that will help make us smarter, healthier, and more productive.The most recent example of artificial intelligence in action is the United States. It has been called the "Jellybean of Artificial Intelligence", and has been used in many fields such as health and medicine.

Artificial Intelligence i

In [21]:
# Question 8.  Implement a simple attention-based encoder-decoder model for
 # English-to-Spanish translation using Tensorflow or PyTorch.

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# ============================================
# 2. Small Toy Dataset
# ============================================

english_sentences = [
    "hello",
    "how are you",
    "i am fine",
    "good morning",
    "thank you"
]

spanish_sentences = [
    "hola",
    "como estas",
    "estoy bien",
    "buenos dias",
    "gracias"
]

# Add start and end tokens
spanish_sentences = ["<start> " + s + " <end>" for s in spanish_sentences]

# ============================================
# 3. Tokenization
# ============================================

eng_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
eng_tokenizer.fit_on_texts(english_sentences)
eng_seq = eng_tokenizer.texts_to_sequences(english_sentences)
eng_max_len = max(len(seq) for seq in eng_seq)
eng_seq = keras.preprocessing.sequence.pad_sequences(eng_seq, maxlen=eng_max_len, padding='post')

spa_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
spa_tokenizer.fit_on_texts(spanish_sentences)
spa_seq = spa_tokenizer.texts_to_sequences(spanish_sentences)
spa_max_len = max(len(seq) for seq in spa_seq)
spa_seq = keras.preprocessing.sequence.pad_sequences(spa_seq, maxlen=spa_max_len, padding='post')

eng_vocab_size = len(eng_tokenizer.word_index) + 1
spa_vocab_size = len(spa_tokenizer.word_index) + 1

print("English Vocabulary Size:", eng_vocab_size)
print("Spanish Vocabulary Size:", spa_vocab_size)

# ============================================
# 4. Build Encoder
# ============================================

embedding_dim = 64
units = 128

class Encoder(keras.Model):
    def __init__(self):
        super().__init__()
        self.embedding = layers.Embedding(eng_vocab_size, embedding_dim)
        self.lstm = layers.LSTM(units, return_sequences=True, return_state=True)

    def call(self, x):
        x = self.embedding(x)
        output, state_h, state_c = self.lstm(x)
        return output, state_h, state_c

# ============================================
# 5. Bahdanau Attention
# ============================================

class BahdanauAttention(layers.Layer):
    def __init__(self):
        super().__init__()
        self.W1 = layers.Dense(units)
        self.W2 = layers.Dense(units)
        self.V = layers.Dense(1)

    def call(self, encoder_outputs, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = self.V(tf.nn.tanh(self.W1(encoder_outputs) + self.W2(hidden_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * encoder_outputs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights



class Decoder(keras.Model):
    def __init__(self):
        super().__init__()
        self.embedding = layers.Embedding(spa_vocab_size, embedding_dim)
        self.lstm = layers.LSTM(units, return_sequences=True, return_state=True)
        self.fc = layers.Dense(spa_vocab_size)
        self.attention = BahdanauAttention()

    def call(self, x, hidden, cell, encoder_outputs):
        context_vector, attention_weights = self.attention(encoder_outputs, hidden)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state_h, state_c = self.lstm(x, initial_state=[hidden, cell])
        output = self.fc(output)
        return output, state_h, state_c


encoder = Encoder()
decoder = Decoder()

optimizer = keras.optimizers.Adam()
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True)



epochs = 300

for epoch in range(epochs):
    total_loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_h, enc_c = encoder(eng_seq)

        dec_input = tf.expand_dims(spa_seq[:, 0], 1)
        dec_hidden, dec_cell = enc_h, enc_c

        for t in range(1, spa_max_len):
            predictions, dec_hidden, dec_cell = decoder(
                dec_input, dec_hidden, dec_cell, enc_output
            )
            loss = loss_object(spa_seq[:, t], predictions[:, 0])
            total_loss += loss
            dec_input = tf.expand_dims(spa_seq[:, t], 1)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(total_loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss.numpy():.4f}")


def translate(sentence):
    seq = eng_tokenizer.texts_to_sequences([sentence])
    seq = keras.preprocessing.sequence.pad_sequences(seq, maxlen=eng_max_len, padding='post')

    enc_output, enc_h, enc_c = encoder(seq)

    dec_input = tf.expand_dims([spa_tokenizer.word_index['<start>']], 0)
    dec_hidden, dec_cell = enc_h, enc_c

    result = ""

    for t in range(spa_max_len):
        predictions, dec_hidden, dec_cell = decoder(
            dec_input, dec_hidden, dec_cell, enc_output
        )
        predicted_id = tf.argmax(predictions[0][0]).numpy()

        if spa_tokenizer.index_word.get(predicted_id) == "<end>":
            break

        result += spa_tokenizer.index_word.get(predicted_id, '') + " "
        dec_input = tf.expand_dims([predicted_id], 0)

    return result.strip()

print("\n===== Translation Results =====")
print("hello →", translate("hello"))
print("thank you →", translate("thank you"))
print("good morning →", translate("good morning"))

English Vocabulary Size: 11
Spanish Vocabulary Size: 11
Epoch 0, Loss: 7.1950
Epoch 50, Loss: 2.0641
Epoch 100, Loss: 0.0602
Epoch 150, Loss: 0.0170
Epoch 200, Loss: 0.0089
Epoch 250, Loss: 0.0057

===== Translation Results =====
hello → hola
thank you → gracias
good morning → buenos dias


In [5]:
# Question 9: Use the following short poetry dataset to simulate poem generation with a
# pre-trained GPT model.

from transformers import pipeline

generator = pipeline("text-generation", model="gpt2")

prompt = """
Roses are red, violets are blue,
Sugar is sweet, and so are you.
The moon glows bright in silent skies,
"""

poem = generator(prompt, max_length=80, num_return_sequences=1)
print(poem[0]['generated_text'])

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Passing `generation_config` together with generation-related arguments=({'max_length', 'num_return_sequences'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Roses are red, violets are blue,
Sugar is sweet, and so are you.
The moon glows bright in silent skies,
And when the sun sets down, the whole world will be happy,
And the starry heavens will be filled with sunshine.
Worth it?
Why do you say it?
Sugar is sweet, and so are you.
The moon glows bright in silent skies,
And when the sun sets down, the whole world will be happy,
And the starry heavens will be filled with sunshine.
Worth it? Why do you say it?
Sugar is sweet, and so are you.
The moon glows bright in silent skies,
And when the sun sets down, the whole world will be happy,
And the starry heavens will be filled with sunshine.
Worth it? Why do you say it?
Sugar is sweet, and so are you.
The moon glows bright in silent skies,
And when the sun sets down, the whole world will be happy,
And the starry heavens will be filled with sunshine.
Worth it? Why do you say it?
Sugar is sweet, and so are you.
The moon glows bright in silent skies,
And when the sun sets down, the whole world wil

Question 10. Imagine you are building a creative writing assistant for a publishing company. The assistant should generate story plots and character descriptions using Generative AI. Describe how you would design the system, including model selection, training data, bias mitigation, and evaluation methods. Explain the real-world challenges you might face.

Answer : If I were building a creative writing assistant for a publishing company, the goal would be to design a system capable of generating:

Original story plots

Detailed character descriptions

Genre-specific narratives

Creative story continuations

The system must be creative, safe, unbiased, and commercially usable.

1. System Architecture Design

The system would consist of the following components:

(A) User Input Interface

Genre selection (Fantasy, Thriller, Romance, Sci-Fi, etc.)

Tone selection (Dark, Humorous, Inspirational)

Target audience (Children, YA, Adult)

Optional keywords (e.g., “dragon”, “detective”, “space colony”)

(B) Prompt Engineering Layer

Converts user input into structured prompts.

Example:

“Generate a 3-paragraph fantasy plot with a strong female protagonist and a moral dilemma.”

(C) Generative Model Layer

Transformer-based Large Language Model (LLM)

Fine-tuned for storytelling tasks

(D) Post-processing & Filtering

Toxicity filtering

Bias detection

Grammar correction

Style consistency check

(E) Human Review (Optional but Recommended)

Editors validate creativity and originality.

2. Model Selection
Recommended Model Types:

GPT-based Transformer Models

Strong text generation capability

Handles long context

Fine-Tuned LLM

Trained on storytelling datasets

Improves narrative coherence

Instruction-Tuned Models

Better control over output style and format

Why Transformer?

Self-attention mechanism captures long-range dependencies.

Generates coherent multi-paragraph text.

Scalable and adaptable.

3. Training Data Strategy
Data Sources:

Public domain novels (e.g., classic literature)

Licensed publishing archives

Screenplay datasets

Character description databases

Genre-specific corpora

Data Preprocessing:

Remove copyrighted non-licensed material

Clean formatting

Remove toxic/harmful content

Tag genre, tone, and character attributes

Data Annotation:

Label by:

Genre

Emotional tone

Character archetype

Narrative structure

4. Bias Mitigation Strategy

Generative AI may inherit bias from training data. To reduce this:

Methods:

Dataset filtering

Remove harmful stereotypes

Balanced representation

Include diverse authors, cultures, and perspectives

Reinforcement Learning with Human Feedback (RLHF)

Human reviewers rate fairness

Bias Detection Tools

Automatic toxicity and stereotype detection APIs

Content moderation layer

Blocks discriminatory or offensive outputs

5. Evaluation Methods
Automatic Metrics:

Perplexity (fluency)

BLEU / ROUGE (structure similarity)

Diversity score (creativity measure)

Human Evaluation:

Creativity rating

Emotional impact

Originality

Commercial suitability

A/B Testing:

Compare model versions

Measure reader engagement

6. Real-World Challenges

Copyright & Ownership

Who owns AI-generated content?

Hallucination

Model may generate inconsistent plot details

Bias & Cultural Sensitivity

Risk of stereotypes

Ethical Misuse

Plagiarism risk

High Computational Cost

Fine-tuning large models is expensive

Controllability

Hard to guarantee exact narrative structure

Editorial Trust

Publishers may resist AI adoption

In [7]:
from transformers import pipeline
generator = pipeline("text-generation", model="gpt2")
prompt_plot = """
Generate a fantasy story plot about a young woman who discovers she can control time.
Include conflict and a moral dilemma.
"""

plot = generator(prompt_plot, max_length=200, num_return_sequences=1)
print("Generated Story Plot:\n")
print(plot[0]['generated_text'])
prompt_character = """
Create a detailed character description for a mysterious detective in a cyberpunk city.
Include personality traits and backstory.
"""

character = generator(prompt_character, max_length=200, num_return_sequences=1)
print("\nGenerated Character Description:\n")
print(character[0]['generated_text'])

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Story Plot:


Generate a fantasy story plot about a young woman who discovers she can control time.
Include conflict and a moral dilemma.
Create a story in which the protagonist is the protagonist of a fantasy story.
Create a story that tells a story about a character who is in love with the hero, and who has a romantic relationship with the hero.
Include the heroine in the story.
Include the heroine in the story.
Add the heroes and villains in the story.
The heroine can be a male or female.
The hero is the hero with the greatest power and can be a powerful villain.
The heroine can be a woman or a man.
The hero cannot be killed or captured, for example, by a human.
The hero can be a slave, a person, a person, a woman or a male slave.
The hero can be a member of the military, the army or a person who is a warrior.
The hero can be a monster, a god, a demon, a demon girl or a god of war, a demon woman or a god of war.
The heroine can be a female or a male.
The hero can be a man 